File này: Với các điểm 3D được chuyển đổi trong ROI (output của file project_to_3d), dựa vào output này,   
ta nhìn vào ảnh point cloud để giữ lại các điểm trong ROI.

Hàm phía dưới là để xử lý với 1 ảnh

In [4]:
# Ý tưởng:
# 1 Load scene point cloud .ply
# 2 load ROI point cloud (Xyz)
# 3 với mỗi điểm ROI -> tìm điểm gần nhất trong scene (KDTree Search)
# 4 Giữ các điểm trong scene thuộc ROI

# filter_ply.py

import numpy as np
import open3d as o3d

class PointCloudFilter:
    def __init__(self, distance_threshold=0.01):
        """
        @param distance_threshold: Ngưỡng để xác định điểm nào thuộc ROI (m)
        """
        self.distance_threshold = distance_threshold

    def filter_scene_with_roi(self, scene_ply_path, roi_pcd):
        """
        @param scene_ply_path: đường dẫn file PLY đầy đủ
        @param roi_pcd: Open3D ROI PointCloud
        @return: filtered_point_cloud (Open3D)
        """
        scene = o3d.io.read_point_cloud(scene_ply_path)

        # KDTree cho scene (tăng tốc độ tìm kiếm lân cận)
        kdtree = o3d.geometry.KDTreeFlann(scene)

        scene_points = np.asarray(scene.points)
        keep_mask = np.zeros(len(scene_points), dtype=bool)

        roi_points = np.asarray(roi_pcd.points)

        for roi_pt in roi_points:
            _, idx, dist = kdtree.search_knn_vector_3d(roi_pt, 1)
            if len(idx) > 0 and dist[0] < self.distance_threshold ** 2:
                [k, idx, dist] = kdtree.search_radius_vector_3d(roi_pt, self.distance_threshold)
                keep_mask[idx] = True

        filtered_scene = scene.select_by_index(np.where(keep_mask)[0])
        return filtered_scene

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Test thử với 1 ảnh

In [5]:
# scene_path = "Public data/Public data train/ply/0002.ply"
# roi_path = "output/filtered_roi_0002.ply"

# roi = o3d.io.read_point_cloud(roi_path)
# filterer = PointCloudFilter(distance_threshold=0.01)

# filtered = filterer.filter_scene_with_roi(scene_path, roi)
# o3d.io.write_point_cloud("output/filtered_scene_0002.ply", filtered)

# print("✅ Filtered PLY saved: output/filtered_scene_0002.ply")

Xử lý trên cả tập data

In [6]:
import os

In [12]:
# 1. Cấu hình
scene_folder = "./Public_data_task_3/Public_data/Public_data_train/ply"       # thư mục chứa các file scene PLY
roi_folder = "./data/roi_pointclouds"          # file ROI PCD
output_folder = "./data/filtered_ply"    # thư mục lưu kết quả
distance_threshold = 0.01

os.makedirs(output_folder, exist_ok=True)

# 2. Load ROI once
# roi_pcd = o3d.io.read_point_cloud(roi_file)

# 3. Tạo object filter
pc_filter = PointCloudFilter(distance_threshold)

# 4. Lặp qua tất cả file PLY trong thư mục
for filename in os.listdir(scene_folder):
    if not filename.endswith(".ply"):
        continue

    scene_path = os.path.join(scene_folder, filename)
    roi_path = os.path.join(roi_folder, filename)

    print(f"\nĐang xử lý: {filename}")
    print(f"Scene path: {scene_path}")
    print(f"ROI path:   {roi_path}")

    if not os.path.exists(scene_path):
        print("Không tìm thấy scene file, bỏ qua.")
        continue
    if not os.path.exists(roi_path):
        print("Không tìm thấy ROI file, bỏ qua.")
        continue

    roi_pcd = o3d.io.read_point_cloud(roi_path)
    if roi_pcd is None or len(roi_pcd.points) == 0:
        print("Không đọc được hoặc ROI rỗng.")
        continue

    filtered_scene = pc_filter.filter_scene_with_roi(scene_path, roi_pcd)

    if filtered_scene is None or len(filtered_scene.points) == 0:
        print("Không có điểm hợp lệ, bỏ qua.")

    output_path = os.path.join(output_folder, filename)
    success = o3d.io.write_point_cloud(output_path, filtered_scene)
    print("Kết quả lưu:", "Thành công" if success else "Thất bại")


Đang xử lý: 0000.ply
Scene path: ./Public_data_task_3/Public_data/Public_data_train/ply\0000.ply
ROI path:   ./data/roi_pointclouds\0000.ply
Không có điểm hợp lệ, bỏ qua.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
Kết quả lưu: Thất bại

Đang xử lý: 0001.ply
Scene path: ./Public_data_task_3/Public_data/Public_data_train/ply\0001.ply
ROI path:   ./data/roi_pointclouds\0001.ply
Không có điểm hợp lệ, bỏ qua.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
Kết quả lưu: Thất bại

Đang xử lý: 0002.ply
Scene path: ./Public_data_task_3/Public_data/Public_data_train/ply\0002.ply
ROI path:   ./data/roi_pointclouds\0002.ply
Không có điểm hợp lệ, bỏ qua.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
Kết quả lưu: Thất bại

Đang xử lý: 0003.ply
Scene path: ./Public_data_task_3/Public_data/Public_data_train/ply\0003.ply
ROI path:   ./data/roi_pointclouds\0003.ply
Không có điểm hợp lệ, bỏ qua.
[Open3D WARNING] Write PLY failed: point cloud has 0 points.
Kế